In [1]:
import pandas as pd
import os

In [2]:
connector = pd.read_csv('../connector/connector.csv')
connector.head()

,sid,course,Diagnosis (Only want Mets),Primary Diagnosis,Age at Diagnosis,Gender,Lesion #,lesion_clinical,mri_type,duration_tx_to_imag (months),Fractions,lesion,date,lesion_rtstruct,lesion_clean,file_name
0,103,1,Mets Ovary,Serous carcinoma,75,Female,2,L Temporal,stable,13.572133,1,Left Temporal,04-18-2014,L temporal,Ltemporal,103_1_Ltemporal
1,103,1,Mets Ovary,Serous carcinoma,75,Female,1,R atrium,stable,13.572133,5,Right Atrium,04-18-2014,R atrium,Ratrium,103_1_Ratrium
2,103,2,Mets Ovary,Serous carcinoma,75,Female,3,Rt Frontal,stable,5.389418,1,Right Frontal,12-18-2014,Rt Frontal,RtFrontal,103_2_RtFrontal
3,103,2,Mets Ovary,Serous carcinoma,75,Female,5,Rt Inf Cerebellar,stable,5.389418,1,Right Inferior Cerebellar,12-18-2014,R Inf Cerebellar,RInfCerebellar,103_2_RInfCerebellar
4,103,2,Mets Ovary,Serous carcinoma,75,Female,6,Rt Lat Cerebellar,stable,5.389418,1,Right Lateral Cerebellar,12-18-2014,Rt Lat Cerebellar,RtLatCerebellar,103_2_RtLatCerebellar


In [11]:
file_list = {'filename':[],'modality':[],'pid':[],'course':[],'lesion':[],'extension':[]}
for filename in os.listdir('dataset'):
    modality,pid,course = filename.split('$',1)[0][:-1].split('_')
    lesion = filename.split('$')[1].replace('_',' ')
    recurrence = int(connector.loc[connector['lesion']==lesion]['mri_type'].values[0]=='recurrence')
    file_list['filename'].append(filename)
    file_list['modality'].append(modality)
    file_list['pid'].append(pid)
    file_list['course'].append(course)
    file_list['lesion'].append(lesion)
    file_list['extension'].append(filename.split('.')[-1])
file_list = pd.DataFrame(file_list)
file_list.head()

,filename,modality,pid,course,lesion,extension
0,dose_234_1_$Right_Frontal$.npy,dose,234,1,Right Frontal,npy
1,struct_257_4_$Right_Parietal$.npy,struct,257,4,Right Parietal,npy
2,struct_103_2_$Right_Frontal$.npy,struct,103,2,Right Frontal,npy
3,mri_224_1_$Right_Cerebellar_Cavity$.npy,mri,224,1,Right Cerebellar Cavity,npy
4,dose_492_1_$Left_Frontal_Cavity$.npy,dose,492,1,Left Frontal Cavity,npy


In [12]:
connector.rename(columns={'sid':'pid'},inplace=True)
connector.pid=connector.pid.astype(int)
connector.course=connector.course.astype(int)
file_list.pid=file_list.pid.astype(int)
file_list.course=file_list.course.astype(int)

In [13]:
connector.shape,file_list.shape

((244, 16), (732, 6))

In [14]:
file_list=file_list.merge(connector[['pid','lesion','course','mri_type']],on=['pid','lesion','course'],how='inner')
file_list.rename(columns={'mri_type':'recurrence'},inplace=True)
file_list.shape

(732, 7)

In [15]:
file_list.columns

Index(['filename', 'modality', 'pid', 'course', 'lesion', 'extension',
       'recurrence'],
      dtype='object')

In [16]:
# Assuming 'mri_type' is the column you want to encode
label_mapping = {'stable': 0, 'recurrence': 1}
file_list['recurrence'] = file_list['recurrence'].map(label_mapping)

In [17]:
file_list.head()

,filename,modality,pid,course,lesion,extension,recurrence
0,dose_234_1_$Right_Frontal$.npy,dose,234,1,Right Frontal,npy,0
1,struct_257_4_$Right_Parietal$.npy,struct,257,4,Right Parietal,npy,0
2,struct_103_2_$Right_Frontal$.npy,struct,103,2,Right Frontal,npy,0
3,mri_224_1_$Right_Cerebellar_Cavity$.npy,mri,224,1,Right Cerebellar Cavity,npy,1
4,dose_492_1_$Left_Frontal_Cavity$.npy,dose,492,1,Left Frontal Cavity,npy,0


In [18]:
file_list.to_csv('file_list.csv',index=False)    